# Setup

On some systems, this cell needs to be run twice to get the correct settings of matplotlib.

In [ ]:
from rrk import *
from scipy import interpolate
import time
import scipy as sp

plt.rc("font", family="serif", size=16.)

# Kepler Problem

In [ ]:
def f(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)
    
    dq1 = p1
    dq2 = p2
    dp1 = -q1 / (abs_q*abs_q*abs_q)
    dp2 = -q2 / (abs_q*abs_q*abs_q)
    return np.array([dq1, dq2, dp1, dp2])

def hamiltonian(w):
    abs2_q = w[0]*w[0] + w[1]*w[1]
    abs2_p = w[2]*w[2] + w[3]*w[3]
    return 0.5 * abs2_p - 1.0 / np.sqrt(abs2_q)

def d_hamiltonian(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    abs_q = np.sqrt(q1*q1 + q2*q2)
    
    dq1 = q1 / (abs_q*abs_q*abs_q)
    dq2 = q2 / (abs_q*abs_q*abs_q)
    dp1 = p1
    dp2 = p2
    return np.array([dq1, dq2, dp1, dp2])

def angular_momentum(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    
    return q1*p2 - q2*p1

def d_angular_momentum(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    
    return np.array([p2, -p1, -q2, q1])

"""
Analytical solution of the Kepler problem, cf.
http://mathworld.wolfram.com/KeplersEquation.html
and
https://matlab-monkey.com/astro/keplerEquation/KeplerEquationPub.html
"""
def u_analytical(t):
    res = fsolve(lambda alpha: alpha - e*np.sin(alpha) - t, 0, xtol=1.e-12)
    alpha = res[0]
    theta = 2.0 * np.arctan(np.sqrt((1+e)/(1-e)) * np.tan(0.5*alpha))
    radius = (1 - e*e) / (1 + e * np.cos(theta))
    q0 = radius * np.cos(theta)
    q1 = radius * np.sin(theta)
    # TODO: using the conservation of angular momentum and energy
    p0 = 0.
    p1 = 0.
    return np.array([q0, q1, p0, p1])

e = 0.5
u0 = np.array([1.0 - e, 0.0, 0.0, np.sqrt((1+e)/(1-e))])

dt = 5.e-2
t_final = 50 * np.pi
erk = ssp33

tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
# tt, uu = convex_relaxed_ERK(erk, dt, f, angular_momentum, d_angular_momentum, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)

plt.close("all");
plt.figure(figsize=(4,4))
plt.plot(uu[0,:], uu[1,:])
plt.axis('equal')

plt.figure()
plt.subplot(211)
H = [hamiltonian(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("Diff. Hamiltonian"); plt.xlim(tt[0], tt[-1]);
plt.subplot(212)
H = [angular_momentum(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0]);
plt.xlabel("$t$"); plt.ylabel("Diff. Ang. Mom."); plt.xlim(tt[0], tt[-1]);

plt.figure()
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors)
plt.xlabel("$t$"); plt.ylabel("Error in $q$")

plt.plot(tt, 8*errors[1]*tt, ":", color="gray")
plt.plot(tt, 8*errors[1]*tt*tt, ":", color="gray")
plt.xlabel("$t$"); plt.ylabel("Error in $q$")
plt.xscale("log"); plt.yscale("log")

### Error Growth

In [ ]:
plt.close("all")


erk = ssp33; dt = 1.0e-2; t_final = 300 * np.pi
plt.figure()
tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=False, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Baseline")

tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Hamiltonian)")

tt, uu = convex_relaxed_ERK(erk, dt, f, angular_momentum, d_angular_momentum, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Angular Momentum)")

plt.plot(tt, 2*errors[1]*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t)$", (3.0e2, 3.0e-6), color="gray")
plt.plot(tt, 600*errors[1]*tt*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^2)$", (1.0e2, 2.0), color="gray")
plt.xlabel("$t$"); plt.ylabel("Error in $q$")
plt.legend(loc="best")
plt.xscale("log"); plt.yscale("log")
plt.savefig("../figures/error_growth__Kepler_ssp33.pdf", bbox_inches="tight")


erk = rk4; dt = 5.e-2; t_final = 300 * np.pi
plt.figure()
tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=False, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Baseline")

tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Hamiltonian)")

tt, uu = convex_relaxed_ERK(erk, dt, f, angular_momentum, d_angular_momentum, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Angular Momentum)")

plt.plot(tt, 0.1*errors[1]*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t)$", (3.0e2, 5.0e-6), color="gray")
plt.plot(tt, 8*errors[1]*tt*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^2)$", (1.0e2, 4.0e-1), color="gray")
plt.xlabel("$t$"); plt.ylabel("Error in $q$")
plt.legend(loc="best")
plt.xscale("log"); plt.yscale("log")
plt.savefig("../figures/error_growth__Kepler_RK4.pdf", bbox_inches="tight")


erk = dp5; dt = 5.e-2; t_final = 300 * np.pi
plt.figure()
tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=False, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Baseline")

tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Hamiltonian)")

tt, uu = convex_relaxed_ERK(erk, dt, f, angular_momentum, d_angular_momentum, u0, t_final, 
                            relaxed=True, method="brentq", newdt=True)
errors = [np.linalg.norm(uu[0:2,i] - u_analytical(tt[i])[0:2]) for i in np.arange(len(tt))]
plt.plot(tt, errors, label="Relaxation (Angular Momentum)")

plt.plot(tt, 0.4*errors[1]*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t)$", (3.0e2, 2.0e-7), color="gray")
plt.plot(tt, 8*errors[1]*tt*tt, ":", color="gray")
plt.annotate(r"$\mathcal{O}(\Delta t^2)$", (1.0e2, 3.0e-3), color="gray")
plt.xlabel("$t$"); plt.ylabel("Error in $q$")
plt.legend(loc="best")
plt.xscale("log"); plt.yscale("log")
plt.savefig("../figures/error_growth__Kepler_dp5.pdf", bbox_inches="tight")

# Outer Solar System

In [ ]:
num_bodies = 6
dim = 3
gravitational_constant = 2.95912208286e-4

def f(w):
    q = w[0:num_bodies*dim].reshape((num_bodies, dim))
    p = w[num_bodies*dim:].reshape((num_bodies, dim))
    v = p / masses[:,None]
    
    dw = np.zeros_like(w)
    dwr = dw.reshape(2, num_bodies, dim)
    dwr[0, :, :] = v
    for i in np.arange(num_bodies):
        for j in np.arange(i):
            diff = q[i, :] - q[j, :]
            abs_diff = np.linalg.norm(diff)
            val = gravitational_constant * masses[i] * masses[j] / (abs_diff*abs_diff*abs_diff) * diff
            dwr[1, i, :] -= val
            dwr[1, j, :] += val
    
    return dw

def hamiltonian(w):
    q = w[0:num_bodies*dim].reshape((num_bodies, dim))
    p = w[num_bodies*dim:].reshape((num_bodies, dim))
    
    energy = 0.
    for i in np.arange(num_bodies):
        energy += 0.5 * np.dot(p[i,:], p[i,:]) / masses[i]

    for i in np.arange(num_bodies):
        for j in np.arange(i):
            diff = q[i, :] - q[j, :]
            abs_diff = np.linalg.norm(diff)
            energy -= gravitational_constant * masses[i] * masses[j] / abs_diff
    
    return energy

def d_hamiltonian(w):
    fw = f(w)
    fwr = fw.reshape(2, num_bodies, dim)
    dw = np.zeros_like(w)
    dwr = dw.reshape(2, num_bodies, dim)
    dwr[0, :, :] = -fwr[1, :, :]
    dwr[1, :, :] =  fwr[0, :, :]
    return dw

masses = np.array([
    1.00000597682, 
    0.000954786104043, 
    0.000285583733151, 
    0.0000437273164546, 
    0.0000517759138449,
    1.0 / 1.3e8
 ])
q0 = np.array([
    0.0, 0.0, 0.0, # Sun with inner planets
     -3.5023653, -3.81698847,  -1.5507963, # Jupiter
      9.0755314, -3.0458353,   -1.6483708, # Saturn
      8.3101420, -16.2901086,  -7.2521278, # Uranus
     11.4707666, -25.7294829, -10.8169456, # Neptune
    -15.5387357, -25.2225594,  -3.1902382, # Pluto
])
p0 = np.array([
    0., 0., 0.,
    0.00565429*masses[1], -0.00412490*masses[1], -0.00190589*masses[1],
    0.00168318*masses[2],  0.00483525*masses[2],  0.00192462*masses[2],
    0.00354178*masses[3],  0.00137102*masses[3],  0.00055029*masses[3],
    0.00288930*masses[4],  0.00114527*masses[4],  0.00039677*masses[4],
    0.00276725*masses[5], -0.00170702*masses[5], -0.00136504*masses[5]
])
u0 = np.concatenate((q0, p0))

erk = ssp22; dt = 200 # 10, 200
t_final = 200000

%time tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, \
                            relaxed=False, method="brentq", newdt=True)

plt.close("all")
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
u = uu.reshape(2, num_bodies, dim, uu.shape[1])
for i in np.arange(num_bodies):
    ax.plot(u[0,i,0,:], u[0,i,1,:], u[0,i,2,:], linewidth=1, linestyle="-")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.dist = 11
plt.savefig("../figures/outer_solar_system__ssp22_orbits.pdf", bbox_inches="tight")

plt.figure()
plt.subplot(111)
H = [hamiltonian(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0], color="black")
plt.xlabel("$t$"); plt.ylabel("$H(u(t)) - H(u(0))$"); plt.xlim(tt[0], tt[-1]);
plt.savefig("../figures/outer_solar_system__ssp22_Hamiltonian.pdf", bbox_inches="tight")

In [ ]:
%time tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, \
                            relaxed=True, method="brentq", newdt=True)

plt.close("all");
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
u = uu.reshape(2, num_bodies, dim, uu.shape[1])
for i in np.arange(num_bodies):
    ax.plot(u[0,i,0,:], u[0,i,1,:], u[0,i,2,:], linewidth=1, linestyle="-")
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_zlabel("$z$")
ax.dist = 11
plt.savefig("../figures/outer_solar_system__ssp22_RRK_orbits.pdf", bbox_inches="tight")
    
plt.figure()
plt.subplot(111)
H = [hamiltonian(uu[:,i]) for i in range(uu.shape[1])]
plt.plot(tt, H - H[0], color="black")
plt.xlabel("$t$"); plt.ylabel("$H(u(t)) - H(u(0))$"); plt.xlim(tt[0], tt[-1]);
plt.savefig("../figures/outer_solar_system__ssp22_RRK_Hamiltonian.pdf", bbox_inches="tight")

# Molecular Dynamics: Frozen Argon Crystal

Cf. Section I.3.2 of Hairer, Lubich, Wanner: Geometric Numerical Integration. Using the RRK approach, the temperature looks much better than for the baseline scheme!

In [ ]:
num_bodies = 7
dim = 2
kB = 1.380658e-23
epsilon = 119.8 * kB
sigma = 0.341
mass = 66.34e-27

def lennard_jones(qi, qj):
    r = np.linalg.norm(qi - qj)
    return 4 * epsilon * ( (sigma/r)**12 - (sigma/r)**6)

def lennard_jones_grad(qi, qj):
    r = np.linalg.norm(qi - qj)
    return 4 * epsilon * ( -12/sigma * (sigma/r)**13 + 6/sigma * (sigma/r)**7) * (qi - qj) / r

def f(w):
    q = w[0:num_bodies*dim].reshape((num_bodies, dim))
    p = w[num_bodies*dim:].reshape((num_bodies, dim))
    v = p / mass
    
    dw = np.zeros_like(w)
    dwr = dw.reshape(2, num_bodies, dim)
    dwr[0, :, :] = v
    for i in np.arange(num_bodies):
        for j in np.arange(i):
            val = lennard_jones_grad(q[i, :], q[j, :])
            dwr[1, i, :] -= val
            dwr[1, j, :] += val
    
    return dw

def hamiltonian(w):
    q = w[0:num_bodies*dim].reshape((num_bodies, dim))
    p = w[num_bodies*dim:].reshape((num_bodies, dim))
    
    energy = 0.
    for i in np.arange(num_bodies):
        energy += 0.5 * np.dot(p[i,:], p[i,:]) / mass

    for i in np.arange(num_bodies):
        for j in np.arange(i):
            energy += lennard_jones(q[i, :], q[j, :])
    
    return energy

def d_hamiltonian(w):
    fw = f(w)
    fwr = fw.reshape(2, num_bodies, dim)
    dw = np.zeros_like(w)
    dwr = dw.reshape(2, num_bodies, dim)
    dwr[0, :, :] = -fwr[1, :, :]
    dwr[1, :, :] =  fwr[0, :, :]
    return dw

def temperature(w):
    q = w[0:num_bodies*dim].reshape((num_bodies, dim))
    p = w[num_bodies*dim:].reshape((num_bodies, dim))
    
    temperature = 0.
    for i in np.arange(num_bodies):
        temperature += np.dot(p[i,:], p[i,:]) / mass
    temperature *= 0.5 / (num_bodies * kB)
    
    return temperature


q0 = np.array([
    0.0, 0.0,
    0.02, 0.39,
    0.34, 0.17,
    0.36, -0.21,
    -0.02, -0.40,
    -0.35, -0.16,
    -0.31, 0.21,
])
p0 = np.array([
    -30.0, -20.0,
    50.0, -90.0,
    -70.0, -60.0,
    90.0, 40.0,
    80.0, 90.0,
    -40.0, 100.0,
    -80.0, -60.0,
]) * mass
u0 = np.concatenate((q0, p0))


In [ ]:
erk = rk4; dt = 8.e-5; t_final = 0.2
plt.close("all")

%time tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, \
                                  relaxed=False, method="brentq", newdt=True)

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6,8))
fig.subplots_adjust(hspace=0)
ax_H = axs[0]
ax_T = axs[1]

H = [hamiltonian(uu[:,i]) for i in range(uu.shape[1])]
ax_H.plot(tt, (H - H[0]) / kB, color="black")
ax_H.set_ylabel("Energy Fluctuation")
ax_H.ticklabel_format(axis="y", style="sci", scilimits=(-1,1))

T = [temperature(uu[:,i]) for i in range(uu.shape[1])]
ax_T.plot(tt, T - T[0], color="black")
ax_T.set_ylabel("Temperature Fluctuation")
ax_T.set_xlabel("$t$"); ax_T.set_xlim(0, t_final);
ax_T.set_ylim([-22, 12]);

plt.savefig("../figures/MD__RK4.pdf", bbox_inches="tight")



%time tt, uu = convex_relaxed_ERK(erk, dt, f, hamiltonian, d_hamiltonian, u0, t_final, \
                                  relaxed=True, method="brentq", newdt=True)

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(6,8))
fig.subplots_adjust(hspace=0)
ax_H = axs[0]
ax_T = axs[1]

H = [hamiltonian(uu[:,i]) for i in range(uu.shape[1])]
ax_H.plot(tt, (H - H[0]) / kB, color="black")
ax_H.set_ylabel("Energy Fluctuation")
ax_H.ticklabel_format(axis="y", style="sci", scilimits=(-1,1))

T = [temperature(uu[:,i]) for i in range(uu.shape[1])]
ax_T.plot(tt, T - T[0], color="black")
ax_T.set_ylabel("Temperature Fluctuation")
ax_T.set_xlabel("$t$"); ax_T.set_xlim(0, t_final);
ax_T.set_ylim([-22, 12]);

plt.savefig("../figures/MD__RK4_RRK.pdf", bbox_inches="tight")